In [24]:
import A1_provider_prep
import A2_other
import A3_analysis
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.wkt
import shared_utils
from calitp.sql import to_snakecase
from shared_utils import geography_utils, utils

In [25]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [26]:
# Load in original geometries of the routes.
one_dist_routes, multi_dist_routes, all_routes = A2_other.find_multi_district_routes()

In [27]:
agencies = all_routes[["agency", "itp_id"]].drop_duplicates().reset_index(drop=True)

In [28]:
len(agencies)

168

In [29]:
agencies

,agency,itp_id
0,AC Transit,4
1,Alhambra Community Transit,6
2,Amador Regional Transit System,11
3,Anaheim Regional Transportation,14
4,Antelope Valley Transit Authority,16
5,Arcadia Transit,17
6,Arcata and Mad River Transit System,18
7,Auburn Transit,23
8,Avocado Heights/Bassett/West Valinda Shuttle,171
9,B-Line,48


In [30]:
data = [
    [177, "the Link Florence-Firestone/Walnut Park", "GTFS but no additional details"],
    [181, "the Link Willowbrook", "GTFS but no additional details"],
    [176, "the Link-Athens", "GTFS but no additional details"],
    [179, "the Link Lennox", "GTFS but no additional details"],
    [178, "the Link King Medical Center", "GTFS but no additional details"],
]
    

In [31]:
def load_gtfs(): 
    
    # Downloaded from Airtable "Organizations"
    # https://airtable.com/appPnJWrQ7ui4UmIl/tblFsd8D5oFRqep8Z/viwVBVSd0ZhYu8Ewm?blocks=hide
    subset_gtfs = ["itp_id", "name", "gtfs_static_status", "gtfs_realtime_status"]
    
    df = to_snakecase(
    pd.read_csv(f"{A1_provider_prep.GCS_FILE_PATH}airtable_organizations.csv"))[subset_gtfs]
    
    df.itp_id = df.itp_id.fillna(0).astype(int)
    
    df.name = df.name.replace(
    {"Eastern Sierra Transit Authority": "Mammoth Lakes Transit System"})
    
    # Consolidate GTFS into one col
    df["gtfs_status"] = df.gtfs_static_status + '/' + df.gtfs_realtime_status
    
    # Drop old cols
    df = df.drop(columns = ["gtfs_static_status", "gtfs_realtime_status"])
    
    # Manually add some agencies that didn't show up
    additional_agencies = pd.DataFrame(data, columns=['itp_id', 'name', 'gtfs_status'])
    df = pd.concat([test, additional_agencies], ignore_index=True)
    
    return df 

In [32]:
test = load_gtfs()

In [35]:
# test

In [21]:
# additional_agencies = pd.DataFrame(data, columns=['itp_id', 'name', 'gtfs_status'])

In [33]:
# pd.concat([test, additional_agencies], ignore_index=True)

In [23]:
# test.sort_values('itp_id')

In [ ]:
    m1 = agencies.merge(gtfs, how="left", on=["itp_id"])

    return m1

In [ ]:
gtfs = to_snakecase(
    pd.read_csv(f"{A1_provider_prep.GCS_FILE_PATH}airtable_organizations.csv")
)[subset_gtfs]

In [ ]:
gtfs.itp_id = gtfs.itp_id.fillna(0).astype(int)

In [ ]:
gtfs.name = gtfs.name.replace(
    {"Eastern Sierra Transit Authority": "Mammoth Lakes Transit System"}
)

In [ ]:
len(gtfs)

In [ ]:
m1 = agencies.merge(gtfs, how="outer", on=["itp_id"], indicator = True)

In [ ]:
m1.loc[m1._merge == "left_only"]

In [ ]:
m1["gtfs_status"] = m1.gtfs_static_status + '/' + m1.gtfs_realtime_status

In [ ]:
# gtfs.name.sort_values().tolist()

In [ ]:
m1.sample()